In [93]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [94]:
#train 폴더와 test 폴더에 있는 positive, negative comments들과 그 라벨을 df에 append한다. (positive = 1, negative = 0)
basepath = 'aclImdb'

labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos','neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)

df.columns = ['review','sentiment']

KeyboardInterrupt: 

In [11]:
#df엔 양성과 음성이 순서대로 나열되어 있으므로 섞어줌
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)

In [13]:
#csv파일로 df를 저장
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [3]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>','',str(text)) # HTML 태그들을 삭제
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|p)', text) #이모티콘들은 감정분석에 도움이 되기 때문에 삭제하지 않음
    text = (re.sub('[W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')) #문자나 숫자가 아닌 것을 모두 삭제 후, 이모티콘을 붙임
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) #헤더는 넘김
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [95]:
df = pd.read_csv('movie_data.csv')

In [59]:
df['review'] = df['review'].apply(preprocessor)

In [4]:
def tokenizer(text):
    return text.split()

In [62]:
#단어를 변하지 않는 기본 형태인 어간으로 바꾸는 어간 추출(stemming), makes -> make
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('what doesn\'t kill you makes you stronger')

['what', "doesn't", 'kill', 'you', 'make', 'you', 'stronger']

In [63]:
#불용어 제거, 불용어는 아주 흔하게 등장하는 단어로 문서의 종류를 구별하는데 별 도움이 안되는 단어들임 (is, and, has ..)
#tf idf vectorizer 에선, 특정한 단어가 많은 문서에서 등장할수록 그 단어의 가중치가 이미 낮아져 있음(term frequency inverse document frequency)
#따라서 불용어 제거는 tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 더 도움됨.


In [100]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [17]:
x_train, x_test, y_train, y_text = train_test_split(df['review'],df['sentiment'], test_size=0.5)

In [66]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [67]:
#GridSeachCV를 통해 최적의 매개변수를 구함
#토크나이저로는 그냥 스플릿하는 토크나이저와 어간을 추출하는 토크나이저르르 비교, 분류기의 규제와 규제 강도도 조절
#또한, idf값도 사용하는 것과 사용하지 않는 것을 비교
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [{'vect__ngram_range':[1,1], 'vect__stop_words':[stop, None], 'vect__tokenizer':[tokenizer, tokenizer_porter],
              'clf__penalty':['l1','l2'], 'clf__C':[1.0,10.0,100.0]},
             {'vect__ngram_range':[(1,1)], 'vect__stop_words':[stop,None], "vect__tokenizer":[tokenizer, tokenizer_porter],
             'vect__use_idf':[False], 'vect__norm':[None], 'clf__penalty':['l1','l2'], 'clf__C':[1.0,10.0,100.0]}
             ]

In [ ]:
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(solver='liblinear', random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(x_train, y_train)

In [ ]:
clf = gs_lr_tfidf.best_estimator
print("CV 정확도 %s 테스트 정확도 %s" %(gs_lr_tfidf.best_score_, clf.score(x_test, y_test)))

In [7]:
#한글을 다룰 땐 표제어를 추출하는 것이 더 좋은 방법임. 이러한 작업을 형태소 분석이라고 부름
#이러한 형태소 분석 작업 konlpy패키지를 통해 할 수 있음.( eg) Okt 클래스(open-korean-text)), 0kt.morphs 메소드를 사용하여 토큰화.
#soynlp도 3개의 형태소 분석기를 제공 LTokenizer는 띄어쓰기를 잘 한 샘플에 알맞음. .tokenize 메소드로 토큰화
#그 외에는 MaxScroeTokenizer, RegexTokenizer가 잘 맞음
#soynlp는 WordExtractor객체를 통해 통계 데이터를 생성하고, 이를 LTokenizer에 매개변수로 줄 수 있음.
#주지 않을 시 말뭉치의 통계를 기반으로 진행함

In [68]:
#gridsearch는 오래 걸리기 때문에 외부 메모리 학습을 사용. 이 방법은 데이터셋을 작은 배치로 나누어 분류기를 점점 학습시킴
#이를 위해 문서에서 샘플을 하나씩 읽어오는 streamdocs함수 사용. 
#외부 메모리 학습에 countvectorizer 와 tfidfvectorizer는 사용할 수 없음. 각각 전체 어휘 사전과 역문서 빈도를 계산해야 함.
#이는 hashingvectorizer를 사용하여 해결 가능. 
next(stream_docs(path = 'movie_data.csv'))

('"Almost in the same league as Yonfan\'s rather atrocious Color Blossoms, Spider Lillies drives the point home that you can make cutting edge cinema without the edge, or much in the way of cutting. It\'s a Taiwanese film, which in this day and age is becoming a novelty at an alarming pace, but more than that tidbit, we can find very little in the way of the noteworthy here.<br /><br />You should know that ostensibly Spider Lillies is also a lesbian-themed story, but in every aspect this is nothing but a plastic ploy to lure in the easily seduced and gullible. In several ways we have here a repeat of fellow recent Taiwan release Eternal Summer. Then it was gay men getting the shortchange treatment, now we have the same thing with women. Zero Chou presents, for your non-existent edification, a tale likely to titillate at most a fifteen year old. They managed some of the art house stance, but in the end this results in a most inane, simply uninteresting foray.<br /><br />The Hong Kong an

In [69]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

In [97]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [98]:
classes = np.array([0,1])
for _ in range(45):
    x_train, y_train = get_minibatch(doc_stream, size=1000) #1000개의 샘플을 읽어옴
    if not x_train:
        break
    x_train = vect.transform(x_train) # 각 샘플을 hashingvectorizer로 벡터화
    clf.partial_fit(x_train,y_train, classes = classes)

In [99]:
x_test, y_test = get_minibatch(doc_stream, size=5000)
x_test = vect.transform(x_test)
print("test precision: %.3f" %clf.score(x_test, y_test))

test precision: 0.825


In [23]:
clf.partial_fit(x_test, y_test)

SGDClassifier(loss='log', max_iter=1, random_state=1)

In [32]:
#토픽 모델링이란 레이블이 없는 텍스트 문서에 토픽을 할당하는 비지도 학습의 클러스터링과 비슷하다고 할 수 있음
#토픽 모델링 기법으로는 잠재 디리클레 할당(latent dirichlet allocation)이 있음. 이는 bow를 입력으로 받고, 
# 이를 문서-토픽 행렬, 단어-토픽 행렬로 분해함, 토픽 개수는 하이퍼파라미터로 수동으로 지정해주어야 함.

In [74]:
from sklearn.feature_extraction.text import CountVectorizer

df['review'].fillna(value='',inplace=True)
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
x = count.fit_transform(df['review'].values)

In [77]:
from sklearn.decomposition import LatentDirichletAllocation 
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')
x_topics = lda.fit_transform(x)

In [84]:
lda.components_.shape #lda의 components에는 각 토픽에 대한 특성의 중요도가 오름차순으로 담겨있음

(10, 5000)

In [82]:
#각 토픽별로 가장 중요한 특성 5개 출력, 토픽 1의 단어들을 보면 호러 영화와 관련이 있음을 알 수 있음
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("topic %d:" % (topic_idx+1), end='')
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

topic 1:horror effects gore dead killer
topic 2:worst guy minutes money stupid
topic 3:war american dvd oscar play
topic 4:action game music original style
topic 5:series comedy episode kids tv
topic 6:role wife performance woman plays
topic 7:script audience poor production worst
topic 8:music black city western white
topic 9:book version role original musical
topic 10:family human beautiful father true


In [91]:
#horror영화의 리뷰일 확률이 가장 높은 리뷰 5개를 출력
horror = x_topics[:,0].argsort()[::-1]
for idx, movie_idx in enumerate(horror[:5]):
    print("\nhorror movie #%d: "%(idx+1))
    print(df['review'][movie_idx][:300],'...')


horror movie #1: 
The Salena Incident is set in Arizona where six death row inmates are being transfered from the state prison for reasons never explained, while driving along the heavily armed prison bus gets a flat & the driver is forced to pull off the road. Then two blonde birds turn up & after seducing the incom ...

horror movie #2: 
***SPOILERS*** ***SPOILERS*** Some bunch of Afrikkaner-Hillbilly types are out in the desert looking for Diamonds when they find a hard mound in the middle of a sandy desert area. Spoilers: The dumbest one starts hitting the mound with a pick, and cracks it open. Then he looks into the hole and stic ...

horror movie #3: 
A group of friends discover gold deep inside an old mine. But by taking the gold and thinking they've hit it big, they awaken a long dead miner who's Hell Bent on protecting his treasure. "Miner's Massacre" is a chintzy b-horror movie in the extreme. You've got all your familiar clichés, your group  ...

horror movie #4: 
A killer,

In [102]:
import pickle
import os
dest = os.path.join('movieclassifier','pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
    
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)